# Experiments with GP pushing and then simulated annealing for finalization

In [9]:
from DLC_for_WBFM.utils.projects.finished_project_data import finished_project_data
import numpy as np
import matplotlib.pyplot as plt
import pickle
import napari, zarr

In [2]:
fname = r"C:\dlc_stacks\Charlie-worm3-new-seg\project_config.yaml"
dat = finished_project_data.load_final_project_data_from_config(fname)

In [3]:
fname = r"C:\dlc_stacks\Charlie-worm3-new-seg\2-training_data\raw\match_dat.pickle"
with open(fname, 'rb') as f:
    matches = pickle.load(f)
    
fname = r"C:\dlc_stacks\Charlie-worm3-new-seg\2-training_data\raw\frame_dat.pickle"
with open(fname, 'rb') as f:
    frames = pickle.load(f)

C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator GaussianProcessRegressor from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


# Visualize the pushed neurons on the next frame

In [44]:
pair = (0, 1)
print(matches[pair])
print(frames[pair[0]], frames[pair[1]])

FramePair with 134 matches 

                ReferenceFrame:
                Frame index: 0 
                Number of neurons: 148 
                ReferenceFrame:
                Frame index: 1 
                Number of neurons: 151 



In [45]:
# Get locations of target neurons
target_zxy = frames[pair[1]].neuron_locs
target_zxy = target_zxy[:, [0,2,1]]

In [46]:
# Get gp pushed locations

gp_zxy = matches[pair].gp_pushed_locations
# Replace z coordinate, which is somehow crazy
gp_zxy[:, 0] = frames[pair[0]].neuron_locs[:, 0]
gp_zxy = gp_zxy[:, [0,2,1]]

affine_zxy = matches[pair].affine_pushed_locations
# Replace z coordinate, which is somehow crazy
# affine_zxy[:, 0] = frames[pair[0]].neuron_locs[:, 0]
affine_zxy = affine_zxy[:, [0,2,1]]

In [47]:
# Split by "has match"
f0_to_f1_matches = {n0: n1 for n0, n1, _ in matches[pair].final_matches}

# F1 matches
has_match_ind = set(f0_to_f1_matches.values())
target_xyz_matched = [row for i, row in enumerate(target_zxy) if i in has_match_ind]
target_xyz_unmatched = [row for i, row in enumerate(target_zxy) if i not in has_match_ind]

# F0 matches
has_match_ind = set(f0_to_f1_matches.keys())

gp_xyz_matched = [row for i, row in enumerate(gp_zxy) if i in has_match_ind]
gp_xyz_unmatched = [row for i, row in enumerate(gp_zxy) if i not in has_match_ind]

affine_xyz_matched = [row for i, row in enumerate(affine_zxy) if i in has_match_ind]
affine_xyz_unmatched = [row for i, row in enumerate(affine_zxy) if i not in has_match_ind]

In [64]:
# Plot using napari
v = napari.view_image(dat.red_data[pair[1]], ndisplay=3)
# v.add_points(gp_xyz_matched, size=3, face_color='green', n_dimensional=True)
v.add_points(gp_xyz_unmatched, size=3, face_color='red', n_dimensional=True)
# v.add_points(affine_xyz_matched, size=3, face_color='green', symbol='x', n_dimensional=True)
v.add_points(affine_xyz_unmatched, size=3, face_color='red', symbol='x', n_dimensional=True)
v.add_points(target_xyz_matched, size=5, symbol='ring', face_color='blue', n_dimensional=True)
v.add_points(target_xyz_unmatched, size=5, symbol='ring', face_color='red', n_dimensional=True)

<Points layer 'Points [3]' at 0x17cdc2573c8>

In [ ]:

# Get gradient images (not in z)
# vol = dat.red_data[pair[1], ...]
# vol_filtered = gaussian(vol, sigma=1)
# thresh = 1.5*np.mean(vol_filtered)
# mask = vol_filtered > thresh
# vol_gradient_x = sobel_h(vol_filtered, mask)
# vol_gradient_y = sobel_v(vol_filtered, mask)

# Actually do it (no annealing)

In [93]:
# from skimage.filters import gaussian, sobel_h, sobel_v
from scipy.spatial import KDTree

In [94]:
pair = (1, 2)

In [95]:
# Get initial affine locations
affine_zxy = matches[pair].affine_pushed_locations
affine_zxy = affine_zxy[:, [0,2,1]]

# Get image
vol = dat.red_data[pair[1], ...]
# vol_filtered = gaussian(vol, sigma=1)

# Get locations of target neurons
target_zxy = frames[pair[1]].neuron_locs
target_zxy = target_zxy[:, [0,2,1]]

In [100]:
# Only check the ones with no match
f0_to_f1_matches = {n0: n1 for n0, n1, _ in matches[pair].final_matches}
has_match_ind = list(f0_to_f1_matches.keys())
no_match_ind = list(range(frames[pair[0]].num_neurons()))
[no_match_ind.remove(i) for i in has_match_ind]

# Set up kdtree to check nearest neighbor
tree = KDTree(target_zxy)

# Parameters
brightness_thresh = 25
distance_thresh = 5.0

In [101]:

for i in no_match_ind:
    zxy = affine_zxy[i, :]
    z, x, y = int(zxy[0]), int(zxy[1]), int(zxy[2])
    brightness = vol[z, x, y]
    nn_dist, _ = tree.query(zxy, k=1)
    
    to_keep = (brightness > brightness_thresh) and (nn_dist > distance_thresh)
    print(to_keep, brightness, nn_dist)
    

False 0 278.634677184079
False 22 6.514810434866421
True 40 8.798306660844792
False 31 3.65074508888712
False 34 4.273267236144742
True 36 9.445009640142503
True 27 12.186874485455414
True 35 7.80057086110525


In [97]:
frames[pair[0]].num_neurons()

151

In [99]:
has_match_ind

[150,
 149,
 147,
 146,
 145,
 144,
 143,
 142,
 141,
 140,
 139,
 138,
 136,
 135,
 133,
 132,
 131,
 130,
 129,
 128,
 127,
 126,
 125,
 121,
 120,
 119,
 118,
 117,
 116,
 115,
 114,
 113,
 112,
 111,
 110,
 109,
 108,
 107,
 106,
 105,
 104,
 103,
 102,
 101,
 100,
 99,
 98,
 97,
 96,
 95,
 94,
 93,
 92,
 91,
 90,
 89,
 88,
 87,
 86,
 85,
 84,
 83,
 82,
 81,
 80,
 79,
 78,
 77,
 75,
 74,
 73,
 72,
 71,
 70,
 69,
 67,
 66,
 65,
 64,
 63,
 62,
 61,
 60,
 59,
 58,
 57,
 56,
 55,
 54,
 53,
 52,
 51,
 50,
 49,
 48,
 47,
 46,
 45,
 44,
 43,
 42,
 41,
 40,
 39,
 38,
 37,
 36,
 35,
 34,
 33,
 32,
 31,
 30,
 29,
 28,
 27,
 26,
 25,
 24,
 23,
 22,
 21,
 20,
 19,
 18,
 17,
 16,
 15,
 14,
 13,
 12,
 11,
 10,
 9,
 8,
 7,
 6,
 5,
 4,
 3,
 2,
 1,
 0]

# Use piecewise affine

In [4]:
pair = (0, 1)
print(matches[pair])
print(frames[pair[0]], frames[pair[1]])

FramePair with 134 matches 

                ReferenceFrame:
                Frame index: 0 
                Number of neurons: 148 
                ReferenceFrame:
                Frame index: 1 
                Number of neurons: 151 



In [5]:
m = matches[pair]
f0, f1 = frames[pair[0]], frames[pair[1]]

In [11]:
f0_to_f1 = m.get_f0_to_f1_dict()

In [47]:
src = np.array([f0.keypoint_locs[k,[2,1]] for k in f0_to_f1.keys()])
dst = np.array([f1.keypoint_locs[v,[2,1]] for v in f0_to_f1.values()])

In [48]:

tform = PiecewiseAffineTransform()
tform.estimate(src, dst)

True

In [49]:
src_transformed = tform.inverse(f0.keypoint_locs[:,[2,1]])
src_transformed = np.where(src_transformed==-1, np.nan, src_transformed)

src_transformed = np.hstack([f0.keypoint_locs[:,[0]], src_transformed]) #[:, [0,2,1]]

In [50]:
# Get locations of target neurons
target_zxy = frames[pair[1]].neuron_locs
target_zxy = target_zxy[:, [0,2,1]]

affine_zxy = matches[pair].affine_pushed_locations
affine_zxy = affine_zxy[:, [0,2,1]]

# Split by "has match"
f0_to_f1_matches = {n0: n1 for n0, n1, _ in matches[pair].final_matches}

# F1 matches
has_match_ind = set(f0_to_f1_matches.values())
target_xyz_matched = [row for i, row in enumerate(target_zxy) if i in has_match_ind]
target_xyz_unmatched = [row for i, row in enumerate(target_zxy) if i not in has_match_ind]

# F0 matches
has_match_ind = set(f0_to_f1_matches.keys())

affine_xyz_matched = [row for i, row in enumerate(affine_zxy) if i in has_match_ind]
affine_xyz_unmatched = [row for i, row in enumerate(affine_zxy) if i not in has_match_ind]

In [52]:
# Plot using napari
v = napari.view_image(dat.red_data[pair[0]], ndisplay=3)
v.add_points(src_transformed, size=3, face_color='red', n_dimensional=True)
v.add_points(affine_xyz_matched, size=3, face_color='green', symbol='x', n_dimensional=True)
# v.add_points(affine_xyz_unmatched, size=3, face_color='red', symbol='x', n_dimensional=True)
v.add_points(target_xyz_matched, size=5, symbol='ring', face_color='blue', n_dimensional=True)
v.add_points(target_xyz_unmatched, size=5, symbol='ring', face_color='red', n_dimensional=True)

<Points layer 'Points [2]' at 0x23cef3a4e08>